In [21]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor,  BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor, RandomForestRegressor
import xgboost 

from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import altair as alt
import datetime as dt

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf  # for data
from pandas_datareader import data as pdr
yf.pdr_override()

In [3]:
def read_data(stock='TSLA', period = "60d", interval = "2m"):
    '''
    reading the data
    '''


    df = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,)


    return df

In [4]:
def read_data_enhanced(stock='TSLA', period = "60d", interval = "2m"):
    '''
    reading the data
    '''


    dfx = pdr.get_data_yahoo(stock,period = period,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = interval,).reset_index()



    df_nasaq = read_data(stock='NDAQ', period = period, interval = interval).reset_index()[['Datetime','Open','Adj Close']]
        
    df_nasaq['fn'] = ((df_nasaq['Open']/df_nasaq['Adj Close'].shift(1))-1)*100
    df_nasaq= df_nasaq[['Datetime','fn']]
    
    
    df_sp = read_data(stock='^GSPC', period = period, interval = interval).reset_index()[['Datetime','Open','Adj Close']]
    df_sp['fs'] = ((df_sp['Open']/df_sp['Adj Close'].shift(1))-1)*100
    df_sp= df_sp[['Datetime','fs']]
    

    dfx = pd.merge(dfx,df_nasaq,on = ['Datetime'], how = 'left')
    
    dfx = pd.merge(dfx,df_sp,on = ['Datetime'], how = 'left')
    
    
        
    return dfx

In [92]:
df1 = read_data(stock='AAPL').reset_index()

[*********************100%***********************]  1 of 1 completed


In [93]:
df2 = df1.copy()

In [89]:
# here is the tail of the df which has the most recent data
df1.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2020-09-28 09:30:00-04:00,502.410004,509.779999,500.010101,509.557007,509.557007,498967
1,2020-09-28 09:32:00-04:00,509.799988,512.000000,507.518494,510.484802,510.484802,236304
2,2020-09-28 09:34:00-04:00,510.299988,515.880005,510.257202,514.332825,514.332825,272169
3,2020-09-28 09:36:00-04:00,514.315002,517.400024,510.920013,511.019989,511.019989,281739
4,2020-09-28 09:38:00-04:00,511.609985,516.919922,511.019989,516.852722,516.852722,202803


In [8]:
df2 = read_data_enhanced(stock='TSLA')
df2.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume,fn,fs
0,2020-08-31 09:30:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-09-28 09:30:00-04:00,424.619995,427.000000,422.000000,425.429993,425.429993,3037163.0,NaN,NaN
2,2020-09-28 09:32:00-04:00,425.470001,428.079987,424.605011,426.649994,426.649994,875668.0,-0.107696,-0.003589
3,2020-09-28 09:34:00-04:00,426.899994,426.899994,423.451904,423.695007,423.695007,667923.0,0.015965,0.009564
4,2020-09-28 09:36:00-04:00,423.869995,425.290009,421.200012,422.690002,422.690002,728853.0,-0.111758,0.000000


### Models

In [7]:
# We have the follwoing model already established

In [10]:
def test_stock(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    df = dfx.copy()
    #df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    #df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))


    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')

    

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [11]:
# now let's see the power of feature engineering:
#we will add the daily return as a feature for this model

In [12]:
def test_stock_w_feature(dfx,intra =True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """
    #print(len(df))


    df = dfx.copy()
    
    # for now the following do not add any predictiv power
    #df['day_of_the_week'] = df['Datetime'].dt.dayofweek
    #df['day_of_month'] = df['Datetime'].apply(lambda x:int(str(x)[8:10]))
    

        


    
    df['f01'] = ((df['Open']/df['Close'].shift(1))-1)*100
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1)
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    #df['f01'] = df['Close']/df['Open']-1


    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower','status_upper']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    ridge_model = Ridge()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__alpha':[0.1,0.25,0.4],
    }



    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    




### Performance Comparision

In [95]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df1, intra = True)

6627   2020-11-12 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.998
error 0.14424442948620564
old price =  119.12000274658203
predicted =  119.12632065017985
difference =  0.006317903597818031


In [96]:
test_stock_w_feature(df2, intra = True)

model score: 0.998
error 0.14351166273049146
old price =  119.12000274658203
predicted =  119.12569560683158
difference =  0.005692860249553178


---

With NASDAQ dataframe :

In [15]:
# let's predict he clost price for TSLA after 5 minutes
test_stock(df2, intra = True)

6627   2020-11-12 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.995
error 0.6827196252036022
old price =  411.8299865722656
predicted =  411.7859513303164
difference =  -0.044035241949245574


In [16]:
test_stock_w_feature(df2, intra = True)

model score: 0.995
error 0.6821938924752808
old price =  411.8299865722656
predicted =  411.7859466321703
difference =  -0.044039940095331076


In [66]:
# Do we same the same gain if use a different modeel: Random Forest ?

In [98]:
def test_stock_grad(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """

    df = dfx.copy()
    df['day_of_the_week'] = df['Datetime'].dt.dayofweek



    
    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    



In [99]:
def test_stock_grad_feature(dfx, intra = True):
    """
    predict the price for the closing price for the bext inerval. 
    
    inputs:
    ------
    - stock: string, symobol for the stick, eg. 'TSLA'
    - intra : Boolean, if True then the period and interval should be 1m, 5m, etc, if False it should be 1d, 7d, etc
    - period : total period
    - interval : interval for the data
    
    """


    df = dfx.copy()
    df['f01'] = ((df['Open']/df['Close'].shift(1))-1)*100


    #print(len(df))
    print(df.tail(1)['Datetime'])
    old = df.tail(1).loc[:,'Open'].to_numpy()[0]
    
    if intra ==True:
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Datetime']=df['Datetime'].map(dt.datetime.toordinal)
            df['SMA_1'] = df['Adj Close'].rolling(window = 26).mean()
            df['SMA_2'] = df['Adj Close'].rolling(window = 50).mean()
            df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
            df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')
    
    elif intra ==False:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Date']=df['Date'].map(dt.datetime.toordinal)
#             df['SMA_1'] = df['Adj Close'].rolling(window = 50).mean()
#             df['SMA_2'] = df['Adj Close'].rolling(window = 100).mean()
#             df['status_sma_lower'] = np.where(df['Adj Close'] < df['SMA_1'],'below_movavg','normal')
#             df['status_sma_upper'] = np.where(df['Adj Close'] > df['SMA_2'],'above_movavg','normal')

     
    df['High'] = df['High'].shift(1)
    df['Low'] = df['Low'].shift(1)
    df['Volume'] = df['Volume'].shift(1) 
    df['Middle Band'] =df['Adj Close'].rolling(window=26).mean()
    df['Upper Band'] = df['Middle Band'] + 1.96*df['Adj Close'].rolling(window=20).std()
    df['Lower Band'] = df['Middle Band'] - 1.96*df['Adj Close'].rolling(window=20).std()
    df['status_lower'] = np.where(df['Adj Close'] < df['Lower Band'],'below_ballinger','normal')
    df['status_upper'] = np.where(df['Adj Close'] > df['Upper Band'],'above_ballinger','normal')
    df = df.dropna()
    df = df.drop(columns = ['Close'])
    y = df['Adj Close']
    X = df.drop(columns =['Adj Close','Volume' ,'Low','High'])
    X_train, X_test, y_train, y_test = train_test_split(X,y,shuffle= False)
        # first transformer for the numeric features
    numeric_features = list(set(list(X_train.columns)) - set(['status_lower','status_upper','status_sma_lower','status_sma_upper']))
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
       ])
    # now a taransformer for the categorical features
    categorical_features = ['status_lower']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    # creating a preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    ridge_model = RandomForestRegressor()
    # include the preprocessor and the model in one pipeline.
    # Now we have a full prediction pipeline.
    reg_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                          ('Regressor', ridge_model)])

    # finally we will pass the pipe line to gridsearchcv to find the optimum paramters for the model
    param_grid = {
        'Regressor__n_estimators':[100],
    }
    search = GridSearchCV(reg_pipeline,param_grid,cv = 5)

    # fitting the model
    search.fit(X_train, y_train)

    # printing the first parameter
    #rint(search.best_params_)
    print("model training score: %.3f" % search.score(X_train, y_train))
    print("model score: %.3f" % search.score(X_test, y_test))
    X_train = X[:-1]
    y_train = y[:-1]
    X_test = X[-1:]
    y_test = y[-1:]
    search.fit(X_train,y_train)
    y_pred = search.predict(X_train)
    print('error',np.sqrt(mean_squared_error(y_train,y_pred)))
    y_test = search.predict(X_test)[0]
    diff = y_test - old
   
    print('old price = ',old)
    print('predicted = ',y_test)
    print('difference = ', diff)
    


In [100]:
## Random forest

In [101]:
# let's predict he clost price for TSLA after 5 minutes
test_stock_grad(df1,intra = True)

6627   2020-11-12 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.997
error 0.059190719945574274
old price =  119.12000274658203
predicted =  119.09996215820313
difference =  -0.020040588378904545


In [102]:
test_stock_grad_feature(df2, intra = True)

6627   2020-11-12 15:58:00-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model training score: 1.000
model score: 0.997
error 0.05902478009220833
old price =  119.12000274658203
predicted =  119.10020515441894
difference =  -0.019797592163087074


> Again we noticed an improved performance when we include this feature

----

In [47]:
### Including NASAQ and others

In [48]:
# let's predict he clost price for TSLA after 5 minutes
test_stock_grad(df2,intra = True)

6550   2020-11-12 13:24:09-05:00
Name: Datetime, dtype: datetime64[ns, America/New_York]
model score: 0.871
error 0.2892603412767126
old price =  413.1860046386719
predicted =  413.27557556152345
difference =  0.08957092285157842


In [ ]:
test_stock_grad_feature(df2, intra = True)